In [48]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import holoviews as hv
import hvplot.pandas

import hvplot.pandas

In [49]:
firstOveralls = pd.read_csv('data/first_overalls.csv')
firstOverallsPlayerData = pd.read_csv('data/first_overalls_player_data.csv')
nhlDraftData = pd.read_csv('data/nhl_draft_data.csv')
nhlPlayerData = pd.read_csv('data/nhl_player_data_2000-2020.csv')
playsGoal = pd.read_csv('data/playsGoalWithFOInfo.csv')
draftPlayerInfo = pd.read_csv('data/allDraftPlayerInfo.csv')
draftPlayerInfoAvg = pd.read_csv('data/draftPlayerInfoAvg.csv')

In [50]:
draftPlayerInfoAvg

,draftYear,round,goals,assists,points,gamesPlayed,numPlayers,avgGoals,avgAssists,avgPoints,avgGamesPlayed,pointsPerGame
0,1970,0,545,884,1429,1281,1,545.000000,884.042000,1429.000000,1281.000000,1.115535
1,1970,1,18,19,37,195,12,1.500000,1.625333,3.083333,16.250000,0.189744
2,1970,2,14,43,57,85,8,1.750000,5.417000,7.125000,10.625000,0.670588
3,1970,3,48,30,78,146,7,6.857143,4.327714,11.142857,20.857143,0.534247
4,1970,4,46,76,122,313,9,5.111111,8.486444,13.555556,34.777778,0.389776
...,...,...,...,...,...,...,...,...,...,...,...,...
425,2023,3,4,5,9,19,6,0.666667,0.875333,1.500000,3.166667,0.473684
426,2023,4,0,0,0,0,7,0.000000,0.042000,0.000000,0.000000,0.000000
427,2023,5,0,0,0,0,6,0.000000,0.042000,0.000000,0.000000,0.000000
428,2023,6,0,0,0,0,4,0.000000,0.042000,0.000000,0.000000,0.000000


In [51]:
#Make dataframe pipeline interactive
#thin playsGoal to only include columns we want to use(xCoord, yCoord, scoringPlayerId, playerName)
# Make a copy of the slice
myDf = playsGoal[['xCoord', 'yCoord', 'scoringPlayerId', 'playerName']].copy()

# Find rows where xCoord is negative
negative_xCoord = myDf['xCoord'] < 0

# Invert yCoord for these rows
myDf.loc[negative_xCoord, 'yCoord'] = myDf.loc[negative_xCoord, 'yCoord'] * -1

# Make xCoord positive only
myDf['xCoord'] = myDf['xCoord'].abs()


# Fill NAs with 0s and create GDP per capita column
myDf = myDf.fillna(0)

# Make DataFrame Pipeline Interactive
myIdf = myDf.interactive()

In [52]:
a = 0
for index, row in playsGoal.iterrows():
    if row.get('playerName') == 'Jack Hughes':
        a = a + 1

print(a)

16248


In [53]:
# Define Panel widgets
player_select = pn.widgets.Select(name='Player', options=myDf['playerName'].unique().tolist())
#remove player_select playerName of 0
player_select.options.remove(0)

player_select

BokehModel(combine_events=True, render_bundle={'docs_json': {'da8812a9-ec94-4002-9585-b9a1aea33254': {'version…

In [54]:
#use allPlayerDraftInfo to make bar chart of 2007 7th round draft picks
#make a copy of the slice

#get all rows with draftYear=2007 and draftRound=7
draftPlayerInfo2007 = draftPlayerInfo.loc[(draftPlayerInfo['draftYear'] == 2007) & (draftPlayerInfo['roundNumber'] == 7)]
draftPlayerInfo2007

#bar chart of 2007 7th round draft picks
draftPlayerInfo2007.hvplot.bar(
    x='playerName',
    y='points',
    rot=90,
    title='2007 7th Round Draft Picks',
    xlabel='Player Name',
    ylabel='Points',
    width=600,
    height=400,
    color='red'
)


:Bars   [playerName]   (points)

In [55]:
xCoord_vs_yCoord_scatterplot_pipeline = (
    myIdf[
        (myIdf.playerName == player_select)
    ]
)


In [56]:
xCoord_vs_yCoord_scatterplot_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'567bbba2-4488-49df-b510-b9f9e99e3bf3': {'version…

In [57]:
xCoord_vs_yCoord_scatterplot = xCoord_vs_yCoord_scatterplot_pipeline.hvplot(x='xCoord', 
                                                                y='yCoord',
                                                                size=80, kind="scatter", 
                                                                alpha=0.2,
                                                                legend=False, 
                                                                height=500,
                                                                xlabel='',
                                                                ylabel='',
                                                                width=500)

image = hv.RGB.load_image("Half_ice_hockey_rink.png", bounds=(-40,-45,112,45))

layout = image*xCoord_vs_yCoord_scatterplot
layout = layout.opts(xaxis=None, yaxis=None, responsive=True)  # Remove axis ticks

xCoord_vs_yCoord_scatterplot_panel = pn.panel(layout, sizing_mode="stretch_both")
xCoord_vs_yCoord_scatterplot_panel
#xCoord_vs_yCoord_scatterplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'27f0856f-14df-4e6d-baae-b3f18842ee9d': {'version…

In [58]:
#Create a table of all the data of the 1st overall draft picks
#Make dataframe pipeline interactive
firstOverallChart = firstOveralls[['playerName', 'triCode', 'position', 'height', 'weight', 'draftYear']].copy()

#reverse order of draftYear
firstOverallChart = firstOverallChart.iloc[::-1]
#initialize a new column for goals, assists, points, and games played
firstOverallChart['goals'] = 0
firstOverallChart['assists'] = 0
firstOverallChart['points'] = 0
firstOverallChart['gamesPlayed'] = 0

#for each row in firstOverallsPlayerData
for index, row in firstOverallsPlayerData.iterrows():
    #add the goals, assists, points, and games played to the firstOverallChart of the player with the same name
    firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'goals'] = firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'goals'] + row['goals']
    firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'assists'] = firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'assists'] + row['assists']
    firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'points'] = firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'points'] + row['points']
    firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'gamesPlayed'] = firstOverallChart.loc[firstOverallChart['playerName'] == row['skaterFullName'], 'gamesPlayed'] + row['gamesPlayed']

firstOverallChart


,playerName,triCode,position,height,weight,draftYear,goals,assists,points,gamesPlayed
53,Connor Bedard,CHI,C,70.0,185.0,2023,9,6,15,16
52,Juraj Slafkovsky,MTL,LW,76.0,229.0,2022,6,9,15,57
51,Owen Power,BUF,D,78.0,213.0,2021,7,40,47,105
50,Alexis Lafrenière,NYR,LW,73.0,193.0,2020,56,55,111,258
49,Jack Hughes,NJD,C,70.0,171.0,2019,99,141,240,267
48,Rasmus Dahlin,BUF,D,74.0,185.0,2018,50,198,248,373
47,Nico Hischier,NJD,C,73.0,179.0,2017,113,183,296,405
46,Auston Matthews,TOR,C,74.0,216.0,2016,335,272,607,548
45,Connor McDavid,EDM,C,72.0,195.0,2015,336,602,938,632
44,Aaron Ekblad,FLA,D,75.0,216.0,2014,114,229,343,666


In [59]:
#create chart based off firstOverallChart
firstOverallChartTable = firstOverallChart.reset_index().hvplot.table(columns=['playerName', 'triCode', 'position', 'height', 'weight', 'draftYear', 'goals', 'assists', 'points', 'gamesPlayed'], title="First Overall Draft Picks", width=1000, height=500, index=False)
firstOverallChartTable

:Table   [playerName,triCode,position,height,weight,draftYear,goals,assists,points,gamesPlayed]

In [60]:
# Convert to DataFrame
firstOverallChartTable_df = firstOverallChartTable.dframe()

column_names = {'playerName': 'Player Name', 'triCode': 'Team', 'position': 'Position', 'height': 'Height', 'weight': 'Weight', 'draftYear': 'Draft Year', 'goals': 'Goals', 'assists': 'Assists', 'points': 'Points', 'gamesPlayed': 'Games Played'}
firstOverallChartTable_df = firstOverallChartTable_df.rename(columns=column_names)


# Then use the pipe method
fSTable = firstOverallChartTable_df.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 8, layout='fit_data_stretch')
fSTable

BokehModel(combine_events=True, render_bundle={'docs_json': {'9179f7fc-ba6b-48dd-9ab1-f3911a8156b6': {'version…

In [61]:
yaxis_draftRoundCompare = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options={
        'Average Goals': 'avgGoals', 
        'Average Assists': 'avgAssists', 
        'Average Points': 'avgPoints', 
        'Average Games Played': 'avgGamesPlayed',
        'Points per Game': 'pointsPerGame'
    },
    button_type='success'
)

yaxis_draftRoundCompare

BokehModel(combine_events=True, render_bundle={'docs_json': {'104ed757-763c-4640-9d2e-843c748ae579': {'version…

In [62]:
DRC_Idf = draftPlayerInfoAvg.interactive()

In [63]:
Round = {
    0: '1st Overall Pick',
    1: '1st Round Picks',
    2: '2nd Round Picks',
    3: '3rd Round Picks',
    4: '4th Round Picks',
    5: '5th Round Picks',
    6: '6th Round Picks',
    7: '7th Round Picks'
}
draftPlayerInfoAvg['Round'] = draftPlayerInfoAvg['round'].map(Round)
draftPlayerInfoAvg

,draftYear,round,goals,assists,points,gamesPlayed,numPlayers,avgGoals,avgAssists,avgPoints,avgGamesPlayed,pointsPerGame,Round
0,1970,0,545,884,1429,1281,1,545.000000,884.042000,1429.000000,1281.000000,1.115535,1st Overall Pick
1,1970,1,18,19,37,195,12,1.500000,1.625333,3.083333,16.250000,0.189744,1st Round Picks
2,1970,2,14,43,57,85,8,1.750000,5.417000,7.125000,10.625000,0.670588,2nd Round Picks
3,1970,3,48,30,78,146,7,6.857143,4.327714,11.142857,20.857143,0.534247,3rd Round Picks
4,1970,4,46,76,122,313,9,5.111111,8.486444,13.555556,34.777778,0.389776,4th Round Picks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,2023,3,4,5,9,19,6,0.666667,0.875333,1.500000,3.166667,0.473684,3rd Round Picks
426,2023,4,0,0,0,0,7,0.000000,0.042000,0.000000,0.000000,0.000000,4th Round Picks
427,2023,5,0,0,0,0,6,0.000000,0.042000,0.000000,0.000000,0.000000,5th Round Picks
428,2023,6,0,0,0,0,4,0.000000,0.042000,0.000000,0.000000,0.000000,6th Round Picks


In [64]:
draftRoundCompare_pipeline = (
    DRC_Idf.groupby(['draftYear', 'Round'])[yaxis_draftRoundCompare].mean().reset_index()
)

In [65]:
#using draftPlayerInfoAvg create a line graph of each round for each year of average goals
draftPlayerInfoAvgFiltered = draftPlayerInfoAvg[draftPlayerInfoAvg['avgAssists'] > 0]
draftPlayerInfoAvgFiltered = draftPlayerInfoAvgFiltered[draftPlayerInfoAvgFiltered['avgGoals'] > 0]
draftPlayerInfoAvgFiltered = draftPlayerInfoAvgFiltered[draftPlayerInfoAvgFiltered['avgPoints'] > 0]
draftPlayerInfoAvgFiltered = draftPlayerInfoAvgFiltered[draftPlayerInfoAvgFiltered['avgGamesPlayed'] > 0]

y_labels = {'avgAssists': 'Average Assists', 'avgGoals': 'Average Goals', 'avgPoints': 'Average Points', 'avgGamesPlayed': 'Average Games Played'}
draftPlayerInfoAvgLine = draftRoundCompare_pipeline.hvplot(
    x='draftYear',
    y=yaxis_draftRoundCompare, 
    line_width=3,
    by='Round', 
    width=1000, 
    height=500, 
    xlabel='Draft Year',
    ylabel='Average of Players',  # Use the value attribute here
    ylim=(0.01, None),
    logy=True,
    )

draftPlayerInfoAvgLine



BokehModel(combine_events=True, render_bundle={'docs_json': {'713e0002-9526-46d7-96c9-6ccb3985802b': {'version…

In [66]:
print(draftRoundCompare_pipeline.columns)

In [67]:
#get years 2005-2010 of round 1 in draftPlayerInfoAvg
draftPlayerInfoAvg2005_2010 = draftPlayerInfoAvg[draftPlayerInfoAvg['draftYear'] >= 2006]
draftPlayerInfoAvg2005_2010

,draftYear,round,goals,assists,points,gamesPlayed,numPlayers,avgGoals,avgAssists,avgPoints,avgGamesPlayed,pointsPerGame,Round
286,2006,0,95,257,352,993,1,95.000000,257.042000,352.000000,993.000000,0.354481,1st Overall Pick
287,2006,1,0,2,2,7,6,0.000000,0.375333,0.333333,1.166667,0.285714,1st Round Picks
288,2006,2,19,28,47,83,6,3.166667,4.708667,7.833333,13.833333,0.566265,2nd Round Picks
289,2006,3,14,29,43,125,6,2.333333,4.875333,7.166667,20.833333,0.344000,3rd Round Picks
290,2006,4,18,61,79,84,6,3.000000,10.208667,13.166667,14.000000,0.940476,4th Round Picks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,2023,3,4,5,9,19,6,0.666667,0.875333,1.500000,3.166667,0.473684,3rd Round Picks
426,2023,4,0,0,0,0,7,0.000000,0.042000,0.000000,0.000000,0.000000,4th Round Picks
427,2023,5,0,0,0,0,6,0.000000,0.042000,0.000000,0.000000,0.000000,5th Round Picks
428,2023,6,0,0,0,0,4,0.000000,0.042000,0.000000,0.000000,0.000000,6th Round Picks


In [68]:
template = pn.template.FastListTemplate(
    title='NHL 1st Overall Draft Picks, How Good Are They?', 
    sidebar=[pn.pane.Markdown("## Evan Switzer"),
             pn.pane.Markdown("#### 2023-11-21"),
             pn.pane.Markdown("## Quick Nav"),
            pn.pane.Markdown("### [First Overall Draft Picks](#first-overall-draft-picks)"),
            pn.pane.Markdown("### [First Overall Draft Pick Goals](#first-overall-draft-pick-goals)"),
            pn.pane.Markdown("### [NHL Draft Round Comparison](#nhl-draft-round-comparison)"),
            pn.pane.Markdown("## About"),
            pn.pane.Markdown("#### This dashboard was created to investigate how good first overall draft picks are in the NHL. It also investigates how good the average player is from each round of the draft."),
            pn.pane.Markdown('## Key Performance Indicators'),
            pn.pane.Markdown('#### The key performance indicators used in this dashboard are goals, assists, points, and games played. They are used to compare first overall draft picks to other draft picks.'),
             pn.pane.Markdown("## Data"),
             pn.pane.Markdown("#### The data came from NHL.com. More specifically, it came from a couple hidden endpoints that the NHL itself uses on their site.")],
    main=[
        pn.pane.Markdown("## NHL First Overall Draft Picks"),
        pn.Row(fSTable, margin=(0,25), width=1000),
        pn.pane.Markdown("## NHL First Overall Draft Pick Goals"),
        pn.Row(pn.Column(pn.panel(layout, sizing_mode="stretch_both", width=700, height = 500)),
               pn.Column(
                   pn.pane.Markdown("## Goal Location Scatterplot"),
                   pn.pane.Markdown("#### This scatterplot is made with the location(x, y) of each goal that players scored. It only contains data from 2009 and onward so many first overall draft picks are not included."),
                   pn.pane.Markdown("#### The pattern of goals scored is very interesting. It shows that most goals are scored within a certain distance of the net and the further away from the net we get the less goals are scored."),
                   pn.pane.Markdown("#### However, some players excel at scoring from certain areas. This can be seen in defensemen who have more of their goals scored from further away, as their position requires them to be further away to be able to defend well."),
                   width=300,
                   )),
        pn.pane.Markdown(" ## NHL Draft Round Comparison"),
        pn.Row(pn.Column(yaxis_draftRoundCompare, draftPlayerInfoAvgLine.panel(height=500), margin=(0,25)))],
    accent_base_color="#298700",
    header_background="#298700",
    sidebar_width = 250,
)
template.show()
template.servable();


Launching server at http://localhost:56869


In [69]:
gretz = {
    'playerName': 'Wayne Gretzky',
    'goals': 894,
    'assists': 1963,
    'points': 2857,
    'gamesPlayed': 1487
}

#add gretz to firstOverallChart
gretz_df = pd.DataFrame([gretz])  # Convert the dictionary to a DataFrame

# Concatenate gretz_df with firstOverallChart
firstOverallChart = pd.concat([firstOverallChart, gretz_df], ignore_index=True)

In [70]:
#remove index 54 from firstOverallChart
firstOverallChart

,playerName,triCode,position,height,weight,draftYear,goals,assists,points,gamesPlayed
0,Connor Bedard,CHI,C,70.0,185.0,2023.0,9,6,15,16
1,Juraj Slafkovsky,MTL,LW,76.0,229.0,2022.0,6,9,15,57
2,Owen Power,BUF,D,78.0,213.0,2021.0,7,40,47,105
3,Alexis Lafrenière,NYR,LW,73.0,193.0,2020.0,56,55,111,258
4,Jack Hughes,NJD,C,70.0,171.0,2019.0,99,141,240,267
5,Rasmus Dahlin,BUF,D,74.0,185.0,2018.0,50,198,248,373
6,Nico Hischier,NJD,C,73.0,179.0,2017.0,113,183,296,405
7,Auston Matthews,TOR,C,74.0,216.0,2016.0,335,272,607,548
8,Connor McDavid,EDM,C,72.0,195.0,2015.0,336,602,938,632
9,Aaron Ekblad,FLA,D,75.0,216.0,2014.0,114,229,343,666


In [ ]:
#calculate points per game
firstOverallChart['pointsPerGame'] = firstOverallChart['points'] / firstOverallChart['gamesPlayed']

In [ ]:
#boxplot of points per game
firstOverallChartBoxPoints = firstOverallChart.hvplot.box(y='points', by='playerName', legend=False, width=1000, height=500, title="Points Per Game by Player")
firstOverallChartBoxPoints
firstOverallChartBoxPPG = firstOverallChart.hvplot.box(y='pointsPerGame', by='playerName', legend=False, width=1000, height=500, title="Points Per Game by Player")
firstOverallChartBoxPPG

:BoxWhisker   [playerName]   (pointsPerGame)

In [ ]:
# Create the dashboard
dashboard = pn.Column(
    firstOverallChartBoxPoints,
    firstOverallChartBoxPPG
)

# Show the dashboard
dashboard.servable("/wayne");
dashboard.show("/wayne")

Launching server at http://localhost:52281
